In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 8
refRatio = 2

deriv = 'CD'
order = 5

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max


In [4]:
def CDStencil(orderIn):
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    loops = int(order / 2)
        
    coefs = np.zeros(loops)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    rCell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    lCell = rCell + 0
    lCell[1::2] = -lCell[1::2]
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(loops)]
    for k in range(loops):
        rCellNew = rCell * deltaXFunc(k + 1)
        lCellNew = lCell * deltaXFunc(k + 1)
        tExp[k] = rCellNew - lCellNew
        if(k == 0):
            coefs[k] = 1
        else:
            scale = tExp[k][(2 * k) + 1] / tExp[k - 1][(2 * k) + 1]
            coefs = scale * coefs
            coefs[k] = (-1)**k

#     DO NOT FORGET TO DIVIDE BY COEFFICIENT!!!!
    stenc[:loops] = -coefs[::-1]
    stenc[loops + 1:] = coefs
    return stenc

In [5]:
def UDStencil(order):
    coefs = np.zeros(order + 1)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    cell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(order + 1)]
    for k in range(order + 1):
        cellNew = cell * deltaXFunc(k)
        tExp[k] = cellNew
    
    tExp = np.asarray(tExp).transpose()
    mat = np.zeros((order, order), float)
    vec = np.zeros(order, float)
    mat[0, :] = tExp[0, :order]
    mat[1:, :] = tExp[2:, :order]
    vec[0] = tExp[0, order]
    vec[1:] = tExp[2:, order]
    vec = -vec
    stenc[order] = 1
    stenc[:order] = LA.inv(mat) @ vec
    val = (tExp @ stenc)[1]
    
#     UNCOMMENT THIS LINE!!!!!
#     stenc = stenc / val



#     print(tExp)
#     print('')
#     print(mat)
#     print('')
#     print(vec)
#     print('')
#     print(val)
#     print('')
#     stenc[:order] = coefs[::-1]
#     stenc[order + 1:] = -coefs
    return stenc

In [6]:
def DDStencil(order):
    stenc = -UDStencil(order)[::-1]
    return stenc

In [26]:
def CentGhost(order, x_0, diff):
    errorLoc = 'ERROR:\nGridTransferTools:\nCentGhost:\n'
    errorMess = ''
    
    degFreed = omega. degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    p = np.where(spots > 0)[0][0]
    q = np.where(spots < 0)[0][0]
    
    ghostCell, n_c, n_f = GTT.GhostCellStencil(order, x_0)
    cells = n_c + n_f
    
    # ADD ERROR CHECKER FOR THE NUMBER OF COURSE AND FINE CELLS ON GRID OF GIVEN SIZE!!!
    

    fullStenc = np.zeros(degFreed, float)
    
    if (x_0 > 0):
        for k in range(cells):
            index = (p - n_f + k + 1) % degFreed
            fullStenc[index] = ghostCell[k]
    else:
        if (x < 0):
            for k in range(cells):
                index = (q - n_c + k + 1) % degFreed
                fullStenc[index] = ghostCell[k]
        else:
            errorMess = 'Argument x_0 cannot be 0!'
    
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    
    return fullStenc

In [35]:
ghostCent = CentGhost(order, -0.5, deriv)
print(ghostCent)
print('')
print(omega.xPatches)

YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5  2. ]
polyInterp: [-0.01785714  0.48214286  1.11607143 -0.88392857  0.36607143 -0.0625    ]

2 4
[ 1.11607143 -0.88392857  0.36607143 -0.0625      0.          0.          0.          0.          0.          0.         -0.01785714  0.48214286]

[[array([0.   , 0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   ])], [array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375, 0.5   ])]]


In [30]:
def MakeSpaceDeriv(omega, order, diff):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
    else:
        orderStenc = order
        if (diff == 'U' or diff == 'UD'):
            stenc = UDStencil(order)
            off = 0
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDStencil(order)
                off = 0
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    p = np.where(spots > 0)[0][0]
    q = np.where(spots < 0)[0][0]
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(-off / 2, off / 2, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)
    
    

    for i in range(orderStenc):
        print(cellFaces[i])
        polyStencSet[i] = CentGhost(order, cellFaces[i], diff)
    
    polyStencSet = np.asarray(polyStencSet)
    print(polyStencSet)
    print('')

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    polyMatD = IMat + 0
    pLow = (p - 1) % degFreed
    pHi = (p + 1) % degFreed
    pAt = p
    polyMatD[pAt, pLow:pHi] = 0.5
    
    
    
    polyMatU = IMat + 0
    qLow = (q + 1) % degFreed
    qHi = (q + 3) % degFreed
    qAt = (q + 1) % degFreed
    polyMatU[qAt, qLow:qHi] = 0.5
    
    mat = np.zeros((degFreed, degFreed), float)
    matLast = mat + 0
    
    for d in range(orderStenc + 1):
        s = off - d
        
        np.fill_diagonal(mat, stenc[d])
        derivMat = np.roll(mat, s, axis = 0)

        if (s > 0):
            polyMat = polyMatD + 0
        if (s < 0):
            polyMat = polyMatU + 0
        
        matThis = derivMat @ polyMat
        print(polyMat)
        print('')
        
#         print(derivMat)
#         print('')
#         matThis = matThis + matLast
        matLast = matThis
        
    derivOp = mat
    return mat

In [31]:
A = MakeSpaceDeriv(omega, order, deriv)

-1.5
YOU'RE USING THE UPDATE.
bounds: [-4.  -3.  -2.  -1.   0.   0.5  1. ]
polyInterp: [ 0.00347222 -0.03373016  0.27876984  0.79960317 -0.06374008  0.015625  ]

4 2
-1.0
YOU'RE USING THE UPDATE.
bounds: [-3.  -2.  -1.   0.   0.5  1.   1.5]
polyInterp: [-1.73472348e-18 -2.77555756e-17  1.00000000e+00  0.00000000e+00  1.11022302e-16 -2.77555756e-17]

3 3
-0.5
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5  2. ]
polyInterp: [-0.01785714  0.48214286  1.11607143 -0.88392857  0.36607143 -0.0625    ]

2 4
0.5
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.5 -1.  -0.5 -0.   1.   2. ]
polyInterp: [-0.0625      0.36607143 -0.88392857  1.11607143  0.48214286 -0.01785714]

2 4
1.0
YOU'RE USING THE UPDATE.
bounds: [-1.5 -1.  -0.5 -0.   1.   2.   3. ]
polyInterp: [-2.77555756e-17  1.11022302e-16  0.00000000e+00  1.00000000e+00 -2.77555756e-17 -1.73472348e-18]

3 3
1.5
YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1.   2.   3.   4. ]
polyInterp: [ 0.015625   -0.06374008  0.79960317  

In [10]:
def CentStenc(order, diff):
    if (diff == 'C' or diff == 'CD'):
        stenc = CDStencil(order)
    else:
        piece = np.zeros(order, float)
        if (diff == 'U' or diff == 'UD'):
            stenc = UDStencil(order)
            stenc = np.append(piece, stenc)
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDStencil(order)
                stenc = np.append(stenc, piece)
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    return stenc

In [11]:
stenc1 = CentStenc(order, 'C')
stenc2 = CentStenc(order, 'U')
stenc3 = CentStenc(order, 'D')
ghostCell = GTT.GhostCellStencil(order, -0.5)
print(stenc1)
print('')
print(stenc2)
print('')
print(stenc3)
print('')
print(ghostCell)
print('')


YOU'RE USING THE UPDATE.
bounds: [-2.  -1.   0.   0.5  1.   1.5  2. ]
polyInterp: [-0.01785714  0.48214286  1.11607143 -0.88392857  0.36607143 -0.0625    ]

[ -1.        7.59375 -60.75      0.       60.75     -7.59375   1.     ]

[  0.           0.           0.           0.           0.         -11.41666667  25.         -25.          16.66666667  -6.25         1.        ]

[ -1.           6.25       -16.66666667  25.         -25.          11.41666667   0.           0.           0.           0.           0.        ]

(array([-0.01785714,  0.48214286,  1.11607143, -0.88392857,  0.36607143, -0.0625    ]), 2, 4)



In [12]:
def CentGhost(order, x_0, diff):
    ghostCell = GTT.GhostCellStencil(order, x_0)
    bounds = GTT.BoundVals(order, x_0)
    lenL = np.shape(bounds[bounds < 0])[0]
    lenR = np.shape(bounds[bounds > 0])[0]
    print(lenL, lenR)
    if (x_0 > 0):
        if (lenR == lenL + 1):
            ghostCent = ghostCell
        else:
            addOn = abs(lenL - lenR + 1)
            piece = np.zeros(addOn, float)
            if (lenL < lenR + 1):
                ghostCent = np.append(piece, ghostCell)
            else:
                ghostCent = np.append(ghostCell, piece)
    else:
        if (x_0 < 0):
            a = 1
        else:
            errorMess = 'Argument x_0 cannot be 0!'
    return ghostCent

In [13]:
centGhost = CentGhost(order, 0.5, deriv)
print(centGhost)

YOU'RE USING THE UPDATE.
bounds: [-2.  -1.5 -1.  -0.5 -0.   1.   2. ]
polyInterp: [-0.0625      0.36607143 -0.88392857  1.11607143  0.48214286 -0.01785714]

YOU'RE USING THE UPDATE.
bounds: [-2.  -1.5 -1.  -0.5 -0.   1.   2. ]


TypeError: '<' not supported between instances of 'tuple' and 'int'

In [ ]:
def SpaceDeriv(omega, order, diff):
    
    # Extract info from omega.
    hs = omega.h
    degFreed = omega.degFreed
    
    # Create empty matrizes to fill later.
    blockMat = np.zeros((degFreed, degFreed), float)
    zeroMat = np.zeros(degFreed, float)
    
    # Interpolate ghost cells.
    ghostStencL = GTT.GhostCellStencil(order, -0.5)
    ghostStencR = GTT.GhostCellStencil(order, 0.5)
    
    # Create common constituents of row pieces.
    rightCell = zeroMat + 0
    leftCell = zeroMat + 0
    ghostL = zeroMat + 0
    ghostR = zeroMat + 0
    ghostL[:order+1] = ghostStencL
    ghostR[:order+1] = ghostStencR
    nrollL = int(np.ceil((order + 1) / 3.))
    nrollR = int(order - nrollL)
    ghostL = np.roll(ghostL, -nrollL)
    ghostR = np.roll(ghostR, -nrollR)
    
    print('HELLO?')
    print(ghostL)
    print(ghostR)
    print('')
    
    # Define distinct row pieces between upwind and center difference.
    if (diff == 'U'):
        rightCell[0] = 1
        leftCell[-1] = 1
        
        cf1v2 = rightCell + 0
        cf2v1 = ghostL
        
        fc1v2 = rightCell + 0
        fc2v1 = zeroMat + 0
        fc2v1[-1] = 0.5
        fc2v1[-2] = 0.5
        fc2v2 = rightCell + 0
        
#         hMat = StepMatrix(omega)
    else:
        if (diff == 'D'):
            rightCell[1] = 1
            leftCell[0] = 1
            
            cf1v2 = zeroMat + 0
            cf1v2[1] = 0.5
            cf1v2[2] = 0.5
            cf2v1 = leftCell + 0
            
            fc1v2 = ghostR
            fc2v1 = leftCell + 0
            
#             hMat = StepMatrix(omega)
        else:
            rightCell[1] = 1
            leftCell[-1] = 1

            cf1v2 = zeroMat + 0
            cf1v2[1] = 0.5
            cf1v2[2] = 0.5
            cf2v1 = ghostL
            
            fc1v2 = ghostR
            fc2v1 = zeroMat + 0
            fc2v1[-1] = 0.5
            fc2v1[-2] = 0.5

#             hMat = 0.5 * StepMatrix(omega)
    
    # Define common row pieces between upwind and center difference.
    cf1v1 = leftCell + 0 # New add.
    cf2v2 = rightCell + 0
    
    fc1v1 = leftCell + 0 # New add.
    fc2v2 = rightCell + 0
    
    # Define rows.
    default = rightCell - leftCell
    cf1 = cf1v2 - cf1v1
    cf2 = cf2v2 - cf2v1
    fc1 = fc1v2 - fc1v1
    fc2 = fc2v2 - fc2v1
    
    # Create vector containing intergrid boundary locations.
    spots = np.roll(hs, -1) - hs
    print('spots is')
    print(spots)
    
    # Fill matrix.
    for k in range(degFreed):
        print('ITERATION ' + str(k) +':')
        if (np.roll(spots, 1 - k)[0] > 0):
            print('CONDITION 1:')
            print(fc2)
            print('')
            row = fc2
        else:
            if (np.roll(spots, -k)[0] < 0): # This one's okay.
                print('CONDITION 2:')
                print(cf1)
                print('')
                row = cf1
                
            else:
                if (np.roll(spots, 1 - k)[0] < 0): # This one's okay.
                    print('CONDITION 3:')
                    print(cf2)
                    print('')
                    row = cf2 #fc1
                    
                else:
                    if (spots[k] > 0): # This one's okay.
                        print('CONDITION 4:')
                        print(fc1)
                        print('')
                        row = fc1 #cf2
                    else:
                        print('CONDITION 5:')
                        print(default)
                        print('')
                        row = default
        blockMat[k, :] = np.roll(row, k)

#     blockMat = hMat @ blockMat

    return blockMat


In [ ]:
derivOp = SpaceDeriv(omega, order, 'CD')

In [ ]:
print(derivOp)

In [ ]:
A = np.arange(64).reshape(8, 8)
print(A)